# Scraping steam to get game description and tags

In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import time

import xml.etree.ElementTree

In [2]:
page = requests.get("http://store.steampowered.com/app/396560") #/Front_Defense/
soup = BeautifulSoup(page.text, 'html.parser') #page.content

## Getting the game description

In [3]:
description = soup.select('[name="Description"]')
print(description[0])

<meta content="“Front Defense” immerses you into the WW2 battlefield, When you shoot, throw grenades, Vive will track your location accurately. Destroy enemy with rocket tube and guns. Fully equipped troops with advanced weapons are waiting for you in the game. Make the best of your weapons to gain the victory." name="Description"/>


In [4]:
type(description[0])

bs4.element.Tag

In [5]:
text = str(description[0])
text[15:-22]

'“Front Defense” immerses you into the WW2 battlefield, When you shoot, throw grenades, Vive will track your location accurately. Destroy enemy with rocket tube and guns. Fully equipped troops with advanced weapons are waiting for you in the game. Make the best of your weapons to gain the victory.'

## Getting the game genres

In [6]:
genres = soup.select('body div [class="app_tag"]')
for genre in genres:
    print(genre.get_text().strip())

Action
Violent
VR
World War II


## Getting a list of all relevant game ids

In [7]:
e = xml.etree.ElementTree.parse('data/api_steampowered.xml')
for atype in e.findall('applist'):
    print(atype)

In [8]:
root = e.getroot()

In [9]:
game_ids = []
for idx in range(len(root[0])):
    game_id = root[0][idx][0].text
    name = root[0][idx][1].text
    game_ids.append(game_id)

## Looping over all games

In [10]:
print('The number of games on steam is: ', len(game_ids))

The number of games on steam is:  44248


In [11]:
game_descriptions = []
game_genres = []
complete_genre_list = []
for game_id in game_ids[:50]:
    # Sleeping the loop between 1 and 2 secs
    sleep_time = np.random.randint(1,3)
    time.sleep(sleep_time)
    
    # Requesting the page
    page = requests.get("http://store.steampowered.com/app/"+str(game_id) )
    soup = BeautifulSoup(page.text, 'html.parser')
    
    # Getting the game description
    description = soup.select('meta[name="Description"]')
    if len(description)>0:
        #print('game_id: ', game_id)
        
        text = str(description[0])
        description = text[15:-22]
        #print(str(description))
        game_descriptions.append(description)
    
        # Getting the genres
        genres = []
        tags = soup.select('body div [class="app_tag"]')
        for item in tags:
            genre = item.get_text().strip()
            genres.append(genre)
            if genre not in complete_genre_list:
                complete_genre_list.append(genre)
        #print(genres)
        game_ids.append(game_id)
        
        game_genres.append(genres)

game_id:  10
game_id:  20
game_id:  30
game_id:  40
game_id:  50
game_id:  60
game_id:  70
game_id:  80
game_id:  100
game_id:  130
game_id:  219
game_id:  220
game_id:  240
game_id:  280
game_id:  300
game_id:  320
game_id:  340
game_id:  360
game_id:  380
game_id:  400
game_id:  410
game_id:  420
game_id:  440
game_id:  500
game_id:  550


In [45]:
thefile = open('data/steam_game_tags.txt', 'w')
for item in thelist:
  print>>thefile, item

NameError: name 'thelist' is not defined

In [44]:
import csv

with open('data/steam_game_tags.txt', 'w') as fp:
    a = csv.writer(fp) #Måske er fejlen her?
    #for description in game_descriptions:
    #    a.writerows(description)
    a.writerows(game_descriptions)

NameError: name 'txt' is not defined

In [39]:
with open('data/steam_game_descriptions.csv', 'w', newline='') as fp:
    fp.writerows(game_descriptions)

AttributeError: '_io.TextIOWrapper' object has no attribute 'writerows'

In [24]:
len(game_descriptions)

25

## Doing one-hot encoding of the genres

In [12]:
tag_to_idx_dict = {}
for idx, tag in enumerate(complete_genre_list):
    tag_to_idx_dict[tag] = idx

In [13]:
tag_matrix = np.zeros( (len(game_descriptions), len(complete_genre_list)) )
for idx, tag_list in enumerate(game_genres):
    #print(tag_list)
    for tag in tag_list:
        #print(tag)
        tag_idx = tag_to_idx_dict[tag]
        tag_matrix[idx, tag_idx] = 1

In [15]:
game_descriptions[:5]

["Play the world's number 1 online action game. Engage in an incredibly realistic brand of terrorist warfare in this wildly popular team-based game. Ally with teammates to complete strategic missions. Take out enemy sites. Rescue hostages. Your role affects your team's success. Your team's success affects your role.",
 'One of the most popular online action games of all time, Team Fortress Classic features over nine character classes -- from Medic to Spy to Demolition Man -- enlisted in a unique style of online team warfare. Each character class possesses unique weapons, items, and abilities, as teams compete online in a variety of game play modes.',
 'Enlist in an intense brand of Axis vs. Allied teamplay set in the WWII European Theatre of Operations. Players assume the role of light/assault/heavy infantry, sniper or machine-gunner class, each with a unique arsenal of historical weaponry at their disposal. Missions are based on key historical operations.',
 "Enjoy fast-paced multipla